In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
start = Time("2023-01-26T05:20:36", scale='utc')
end = Time("2023-01-26T05:22:36", scale='utc')

In [ ]:
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition', 'timestamp'],  start, end)    


In [ ]:
len(az)

In [ ]:
plt.subplot(1,2,1)
az['actualPosition'].plot(legend=True, color='red')
plt.subplot(1,2,2)
el['actualPosition'].plot(legend=True, color='green')

In [ ]:
az_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['azimuth', 'timestamp'],  start, end)
el_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['elevation', 'timestamp'],  start, end)    


In [ ]:
len(az_track)

In [ ]:
plt.subplot(1,2,1)
az_track['azimuth'].plot(legend=True, color='red')
plt.subplot(1,2,2)
el_track['elevation'].plot(legend=True, color='green')

In [ ]:
start = Time("2023-01-26T05:21:05", scale='utc')
end = Time("2023-01-26T05:21:10", scale='utc')

In [ ]:
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition', 'timestamp'],  start, end)    


In [ ]:
len(az)

In [ ]:
az_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['azimuth', 'taiTime'],  start, end)
el_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['elevation', 'taiTime'],  start, end)    


In [ ]:
len(az_track)

In [ ]:
azs = az_track.values[:,0]
times = az_track.values[:,1]

for i in range(1,len(az_track)):
    if abs(azs[i] - azs[i-1]) > 0.5:
        shift = times[i]
        break
    
shift_time =  Time(shift, format='unix_tai', scale='utc').isot   
               

In [ ]:
shift_time

In [ ]:
plt.subplot(1,2,1)
ax1 = az['actualPosition'].plot(legend=True, color='red')
ax1.axvline(shift_time, color="black", linestyle="--", label="Start slew")
plt.subplot(1,2,2)
ax2 = el['actualPosition'].plot(legend=True, color='green')
ax2.axvline(shift_time, color="black", linestyle="--", label="Start slew")


In [ ]:
# Plotting the mount plots
%matplotlib inline
track_counter = 4 # This chooses one of the tracking times
start = Time("2023-01-26T05:21:07.5", scale='utc')
end = Time("2023-01-26T05:21:45", scale='utc')
label = ''.join(filter(lambda x: x not in "-:.", str(start.isot)))
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition', 'timestamp'],  start, end)    
# Calculate the tracking errors
az_vals = np.array(az.values[:,0])
el_vals = np.array(el.values[:,0])
times_az = az.values[:,1]
times_el = el.values[:,1]
# The fits are much better if the time variable
# is centered in the interval
time_delta_az = times_az[int(len(az.values) / 2)]
time_delta_el = times_el[int(len(el.values) / 2)]
fit_times_az = [(times_az[i]-time_delta_az) for i in range(len(times_az))]
fit_times_el = [(times_el[i]-time_delta_el) for i in range(len(times_el))]

# Fit with a polynomial
az_fit = np.polyfit(fit_times_az[20:-1], az_vals[20:-1], 4)
el_fit = np.polyfit(fit_times_el[20:-1], el_vals[20:-1], 4)
az_model = np.polyval(az_fit, fit_times_az)
el_model = np.polyval(el_fit, fit_times_el)

# Errors in arcseconds
az_error = (az_vals - az_model) * 3600
el_error = (el_vals - el_model) * 3600
for i in range(len(az_error)):
    if (abs(az_error[i]) < 0.005) and (abs(el_error[i]) < 0.005):
        settled = fit_times_az[i]
        break
settled_time =  Time(settled + time_delta_az, format='unix_tai', scale='utc').isot   
print(settled)

# Calculate RMS
az_rms = np.sqrt(np.mean(az_error * az_error))
el_rms = np.sqrt(np.mean(el_error * el_error))

# Calculate Image impact RMS
# We are less sensitive to Az errors near the zenith
image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
image_el_rms = el_rms 

fig = plt.figure(figsize = (8,8))
plt.subplots_adjust(wspace=0.3, hspace=0.5)
plt.suptitle(f"MT Mount Jitter - 20221214", fontsize = 18)
plt.subplot(2,2,1)
ax1 = az['actualPosition'].plot(legend=True, color='red')
ax1.axvline(settled_time, color="black", linestyle="--", label="Settled")
ax1.set_title("Azimuth")
ax1.set_ylabel("Degrees")
plt.subplot(2,2,2)
ax3 = el['actualPosition'].plot(legend=True, color='green')
ax3.axvline(settled_time, color="black", linestyle="--", label="Settled")
ax3.set_title("Elevation")
ax3.set_ylabel("Degrees")

plt.subplot(2,2,3)
plt.plot(fit_times_az, az_error, color='red')
plt.plot([settled, settled],[-0.5,0.5], ls='--', color='black')
plt.title(f"Azimuth RMS error = {az_rms:.2f} arcseconds\n"
          f"  Image RMS error = {image_az_rms:.2f} arcseconds", fontsize=10)
plt.ylim(-0.5,0.5)
plt.xticks([])
plt.ylabel("ArcSeconds")
plt.subplot(2,2,4)
plt.plot(fit_times_el, el_error, color='green')
plt.plot([settled, settled],[-0.5,0.5], ls='--', color='black')
plt.title(f"Elevation RMS error = {el_rms:.2f} arcseconds\n"
          f"  Image RMS error = {image_el_rms:.2f} arcseconds", fontsize=10)
plt.ylim(-0.5,0.5)
plt.xticks([])
plt.ylabel("ArcSeconds")
#plt.savefig(f"/home/craiglagegit/DATA/MT_Mount_Jitter_{label}.pdf")



In [ ]:
# Now let's try to expand it to the whole run
start = Time("2023-01-26T06:51:00", scale='utc')
end = Time("2023-01-26T09:50:00", scale='utc')
#start = Time("2023-01-26T08:09:00", scale='utc')
#end = Time("2023-01-26T08:51:00", scale='utc')

az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition', 'timestamp'],  start, end)    

az_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['azimuth', 'taiTime'],  start, end)
el_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['elevation', 'taiTime'],  start, end)   
print(len(az_track), len(el_track))

In [ ]:
azPos = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthInPosition', \
                                            ['inPosition', 'private_efdStamp'],  start, end)
azPos = azPos[azPos['inPosition']] # Select only the True values
elPos = await client.select_time_series('lsst.sal.MTMount.logevent_elevationInPosition', \
                                            ['inPosition', 'private_efdStamp'],  start, end)
elPos = elPos[elPos['inPosition']] # Select only the True values
print(len(azPos), len(elPos))

In [ ]:
azs = az_track.values[:,0]
els = el_track.values[:,0]
times = az_track.values[:,1]
start_slew_times_1 = []
slew_dist_1 = []
slew_times_1 = []
for i in range(1,len(az_track)):
    az_shift = abs(azs[i] - azs[i-1])
    el_shift = abs(els[i] - els[i-1])
    if (az_shift > 0.1) or (el_shift > 0.1):
        start_slew_times_1.append(times[i])
        az_shift_mod = az_shift * np.cos(els[i]*np.pi/180.0)
        shift = np.sqrt(el_shift*el_shift + az_shift_mod*az_shift_mod)
        slew_dist_1.append(shift)
print(len(start_slew_times_1))


inPos_1 = []
azPosValues = azPos.values[:,1]
elPosValues = elPos.values[:,1]

for i in range(len(azPos)):
    if azPosValues[i] > elPosValues[i]:
        inPos_1.append(azPosValues[i])
    else:
        inPos_1.append(elPosValues[i])

print(len(inPos_1))
# Now pair them up
pairMin = 8.0
pairMax = 17.0

start_slew_times = []
slew_dist = []
slew_times = []
inPos = []

for i in range(len(start_slew_times_1)):
    for j in range(len(inPos_1)):
        deltaT = inPos_1[j] - start_slew_times_1[i] 
        if deltaT > pairMin and deltaT < pairMax:
            inPos.append(inPos_1[j])
            start_slew_times.append(start_slew_times_1[i])
            slew_times.append(deltaT)
            slew_dist.append(slew_dist_1[i])
        
print(len(inPos), len(start_slew_times), len(slew_times), len(slew_dist))

In [ ]:
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.hist(slew_times)
plt.xlabel("Slew time (seconds)")
plt.xlim(10.0, 20.0)
plt.subplot(1,2,2)
plt.scatter(slew_dist, slew_times)
plt.ylabel("Slew and settle time(sec)")
plt.xlabel("Slew distance (degrees)")

In [ ]:
for i, start_slew_time in enumerate(start_slew_times):
    print(i, start_slew_times[i+1]-start_slew_time, slew_dist[i])
               

In [ ]:
delta1 = 0
delta = 120
azPos_values = azPos.values[:,1]
plt.subplot(1,1,1)
ax1 = az['actualPosition'].plot(color='red')
for start_slew_time in start_slew_times:
    ss_time = Time(start_slew_time, format='unix_tai', scale='utc').isot  
    ax1.axvline(ss_time, color="black", linestyle="--")
for inP in inPos:
    ip_time = Time(inP, format='unix_tai', scale='utc').isot  
    ax1.axvline(ip_time, color="blue", linestyle="--")
ax1.set_xlim((start+TimeDelta(delta1, format='sec')).isot, (start+TimeDelta(delta, format='sec')).isot)
ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
#ax1.set_ylim(-80, -65)
ax1.legend()

In [ ]:
plotAz = az[(az['timestamp'] > inPos[index]) & (az['timestamp'] < start_slew_times[index+1])]
print(len(plotAz))

In [ ]:
# Plotting the mount plots

index = 69
plotAz = az[(az['timestamp'] > inPos[index]) & (az['timestamp'] < start_slew_times[index])]
plotEl = el[(el['timestamp'] > inPos[index]) & (el['timestamp'] < start_slew_times[index])]
ss_time = Time(start_slew_times[index], format='unix_tai', scale='utc').isot
ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot


# Calculate the tracking errors
az_vals = np.array(plotAz.values[:,0])
el_vals = np.array(plotEl.values[:,0])
times_az = plotAz.values[:,1]
times_el = plotEl.values[:,1]
# The fits are much better if the time variable
# is centered in the interval
time_delta_az = times_az[int(len(plotAz.values) / 2)]
time_delta_el = times_el[int(len(plotEl.values) / 2)]
fit_times_az = [(times_az[i]-time_delta_az) for i in range(len(times_az))]
fit_times_el = [(times_el[i]-time_delta_el) for i in range(len(times_el))]

# Fit with a polynomial
az_fit = np.polyfit(fit_times_az, az_vals, 4)
el_fit = np.polyfit(fit_times_el, el_vals, 4)
az_model = np.polyval(az_fit, fit_times_az)
el_model = np.polyval(el_fit, fit_times_el)

# Errors in arcseconds
az_error = (az_vals - az_model) * 3600
el_error = (el_vals - el_model) * 3600

# Calculate RMS
az_rms = np.sqrt(np.mean(az_error * az_error))
el_rms = np.sqrt(np.mean(el_error * el_error))

# Calculate Image impact RMS
# We are less sensitive to Az errors near the zenith
image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
image_el_rms = el_rms 

fig = plt.figure(figsize = (8,8))
plt.subplots_adjust(wspace=0.3, hspace=0.5)
plt.suptitle(f"MT Mount Jitter - {ip_time}", fontsize = 18)
plt.subplot(2,2,1)
ax1 = plotAz['actualPosition'].plot(legend=True, color='red')
ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
ax1.set_title("Azimuth")
ax1.set_ylabel("Degrees")
ax1.legend()
plt.subplot(2,2,2)
ax3 = plotEl['actualPosition'].plot(legend=True, color='green')
ax3.axvline(ss_time, color="black", linestyle="--", label="Start slew")
ax3.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
ax3.set_title("Elevation")
ax3.set_ylabel("Degrees")
ax3.legend()
plt.subplot(2,2,3)
plt.plot(fit_times_az, az_error, color='red')
plt.title(f"Azimuth RMS error = {az_rms:.3f} arcseconds\n"
          f"  Image RMS error = {image_az_rms:.3f} arcseconds", fontsize=10)
plt.ylim(-0.2,0.2)
plt.xticks([])
plt.ylabel("ArcSeconds")
plt.subplot(2,2,4)
plt.plot(fit_times_el, el_error, color='green')
plt.title(f"Elevation RMS error = {el_rms:.3f} arcseconds\n"
          f"  Image RMS error = {image_el_rms:.3f} arcseconds", fontsize=10)
plt.ylim(-0.2,0.2)
plt.xticks([])
plt.ylabel("ArcSeconds")
#plt.savefig(f"/home/craiglagegit/DATA/MT_Mount_Jitter_{label}.pdf")

